# **Import Pakages**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# **Data Scraping Functions**

In [42]:
# function to extract title
def get_title(soup):
  try:

    # outer tag object
    title =soup.find("span", attrs = {"id": 'productTitle'})

    # inner nevitable string object
    title_value = title.text

    # title as a string value
    title_string = title_value.strip()

  except AttributeError:
    title_string = ""

  return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs = {"class": 'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs = {"class": 'a-offscreen'})

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

  try:

    rating = soup.find("span", attrs = {"class": 'a-size-medium a-color-base'}).string.strip()

  except AttributeError:

    try:
            rating = soup.find("span", attrs = {"class": 'a-size-medium a-color-base'}).string.strip()

    except:

      rating = ""

  return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs = {"id": 'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count


# Function to extract Brand
def get_brand(soup):

  try:

    brand = soup.find("td", attrs = {"class": 'a-size-base prodDetAttrValue'}).string.strip()

  except AttributeError:

    try:
            brand = soup.find("td", attrs = {"class": 'a-size-base prodDetAttrValue'}).string.strip()

    except:

      brand = ""

  return brand

# **HTML Request & Data Collection**

In [43]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=laptop&crid=QLFZXQW4D89T&sprefix=lapto%2Caps%2C301&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"brand":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['brand'].append(get_brand(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

# **Data Exploration**

In [44]:
amazon_df

,title,price,rating,reviews,brand
0,"HP Laptop 15s, 12th Gen Intel Core i3, 15.6-in...","₹37,012.00",4 out of 5,605 ratings,‎HP
1,(Refurbished) HP C640 10th Gen Intel Core i5 T...,"₹17,999.00",3.1 out of 5,8,‎HP
2,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...","₹25,990.00",3.9 out of 5,"1,384 ratings",‎ASUS
3,"HP Laptop 15s, AMD Ryzen 3 5300U, 15.6-inch (3...","₹32,610.00",4.1 out of 5,"1,267",‎HP
4,Lenovo IdeaPad Slim 3 Intel Core i5-12450H 15....,"₹53,490.00",4 out of 5,403 ratings,‎Lenovo
5,"HP Laptop 14s, AMD Ryzen 5 5500U, 14-inch (35....","₹41,490.00",4.2 out of 5,"2,266 ratings",‎HP
6,"ASUS Vivobook 16 (2023), Intel Core i9-13900H ...","₹94,990.00",4 out of 5,132,‎ASUS
7,"ASUS Vivobook 16X, AMD Ryzen 7 5800HS, 16"" (40...","₹58,990.00",4.2 out of 5,"1,179 ratings",‎ASUS
8,Lenovo IdeaPad Slim 3 Intel Core i3 12th Gen 1...,"₹34,390.00",4.1 out of 5,815,‎Lenovo
9,(Refurbished) Lenovo ThinkPad 6th Gen Intel Co...,"₹19,999.00",4.5 out of 5,61 ratings,‎Lenovo
